In [1]:
from basic_tools import *
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
h2_total=pd.read_csv(h2_total_par_filtered_path,index_col='phenotype')
h2_total.shape

(497, 7622)

In [3]:
data_merged=phenotypes_par_filtered.merge(h2_total[h2_total.columns[h2_total.columns.str.contains('uni')]],on='phenotype')
data_merged.columns=data_merged.columns.str.replace('uni.','')

In [4]:
#data_merged['description']=data_merged.apply(lambda x: '<a href="/phenotype/{}">'.format(x.name)+x['description']+'</a>',axis=1)
data_merged['UK Biobank Data Showcase Link']=data_merged['UK Biobank Data Showcase Link']#.map(lambda x: '<a href="{}">{}</a>'.format(x,x) if type(x)==str else '')
#=data_merged.apply(lambda x: '<a href="{}">'.format(x['UK Biobank Data Showcase Link'])+x['UK Biobank Data Showcase Link']+'</a>' if type(x),axis=1)

data_merged['-log10(h2_p)']=-np.log10(data_merged['Total h2 p']).round(2)
data_merged['prev(%)']=(data_merged['n_cases']/(data_merged['n_cases']+data_merged['n_controls'])*100).round(2)
data_merged['Mean Chi^2']=data_merged['Mean Chi^2'].round(2)
data_merged['Lambda GC']=data_merged['Lambda GC'].round(2)
data_merged['Intercept']=data_merged['Intercept'].round(2)

In [6]:
data_merged.columns

Index(['PHESANT_transformation', 'Sex', 'description', 'n_cases', 'n_controls',
       'n_missing', 'n_non_missing', 'notes', 'source', 'variable_type',
       'Phenotype Code', 'UK Biobank Data Showcase Link', 'File',
       'wget command', 'Dropbox File', 'filename', 'Total Observed scale h2',
       'Total Observed scale h2_se', 'Total Liability scale h2',
       'Total Liability scale h2_se', 'Lambda GC', 'Mean Chi^2', 'Intercept',
       'Intercept_se', 'Ratio', 'Total h2 z', 'Total h2 p', '-log10(h2_p)',
       'prev(%)'],
      dtype='object')

In [7]:
data_merged=data_merged[['description', 'Sex', 
       'Total Observed scale h2', 'Total Observed scale h2_se','Total Liability scale h2', 'Total Liability scale h2_se','-log10(h2_p)',
        'Intercept', 'Intercept_se','Ratio','Lambda GC','Mean Chi^2',  
        'prev(%)','n_missing', 'n_non_missing', 'n_cases', 'n_controls',               
       'UK Biobank Data Showcase Link', 'File',# 'Dropbox File',
        'source', 'variable_type','PHESANT_transformation','notes'
        ]]

data_merged.to_csv("phenotype_info.csv")

In [8]:
suffix_bp_list_power="bp300 bp128 bp64 bp32 bp16 bp8".split()

In [22]:
#regression_result_bp_list_dict['23115_irnt']

In [26]:
for idx,suffix in enumerate(suffix_bp_list_power):
    regression_result_bp_list_dict=dict([make_regression_result_list(h2_total,phenotype,suffix_bp_list_power,False) for phenotype in phenotypes_par_filtered.index])

    h2_value_df=pd.DataFrame([[phenotype]+(regression_result_list[idx]['h2']).tolist() for phenotype,regression_result_list in regression_result_bp_list_dict.items()])
    h2_value_df=h2_value_df.set_index(0)
    h2_value_df.columns=regression_result_bp_list_dict['23115_irnt'][idx]['category'].apply(category_to_format,args=('chr_bp',))

    file_suffix=suffix[2:]+'Mb' if suffix!='bp300' else 'chr'
    h2_value_df.to_csv("phenotype_info_{}.csv".format(file_suffix))